In [1]:
%reload_ext autoreload
%autoreload 2

# Load the data

In [2]:
from result_saver import SaverProvider
provider = SaverProvider()

In [8]:
from Scratch import metadata_loader
import numpy as np
from tqdm import tqdm
from time import sleep

DEVICE = "ibm_sherbrooke"
LOGICAL = str(0)
XBASIS = True
ROUNDS = 50

state = "X" if XBASIS else "Z"
state += LOGICAL
print(f"state = {state}")


# Load the metadata
while True:
    try:
        md = metadata_loader(True, True)
        break
    except:
        sleep(5)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["code"] == "RepetitionCodeCircuit"]
md = md[md["descr"] == 'subset RepCodes']
md = md.dropna(subset=["rounds"])
md = md[md["meas_level"] == 1]
md['rounds'] = md['rounds'].astype(int)
md['distance'] = md['distance'].astype(int)

md = md[md["backend_name"] == DEVICE]
md = md[md["logical"] == LOGICAL]
md = md[md["xbasis"] == XBASIS]
md = md[md["rounds"] == ROUNDS]



md = md[0:1]
print("shape:", md.shape)
md

state = X0
shape: (1, 18)


,creation_date,notebook_name,backend_name,job_id,tags,meas_level,shots,num_qubits,job_status,execution_date,code,distance,rounds,logical,descr,resets,xbasis,path_info
5086,2024-03-23 14:11:49.436000+01:00,Sherbrooke_RepCodes_jobs,ibm_sherbrooke,cqzda58czq6g0081he8g,"[Subset 52, 50 rounds, 0 log, xbasis=True]",1.0,1507.0,NaN,JobStatus.DONE,2024-03-23 15:41:07.958655+01:00,RepetitionCodeCircuit,52,50,0,subset RepCodes,NaN,True,"{'mean_gate_error': 0.007733045348037, 'min_ga..."


# Group job ids by closest calibration date

In [9]:
from datetime import datetime
from Scratch import find_closest_calib_jobs
import pandas as pd

DOUBLE_MSMT = False # to get also 03-16

jobs_by_calibration_date = {}
for index, row in md.iterrows():
    job_id = row['job_id']

    while True:
        try:
            _, _, calib_creation_date = find_closest_calib_jobs(tobecalib_job=job_id, verbose=False, double_msmt=DOUBLE_MSMT)
            break
        except:
            sleep(5)

    if calib_creation_date not in jobs_by_calibration_date.keys():
        jobs_by_calibration_date[calib_creation_date] = [job_id]
    else:
        jobs_by_calibration_date[calib_creation_date].append(job_id)

# Takes 15s

In [10]:
print(jobs_by_calibration_date)
print()
print(f"num of calibrations: {len(jobs_by_calibration_date)}")
print(f"num of jobs per calibration: {([len(jobs) for jobs in jobs_by_calibration_date.values()])}")

{datetime.datetime(2024, 3, 23, 13, 12, 1, 375000, tzinfo=datetime.timezone.utc): ['cqzda58czq6g0081he8g']}

num of calibrations: 1
num of jobs per calibration: [1]


# Decode data

In [12]:
d = md["distance"].values[0]

distances = np.arange(7, d+1, 4)
distances = np.arange(7, 39, 4)
distances = distances[::-1]

distances

array([35, 31, 27, 23, 19, 15, 11,  7])

In [15]:
import json
import os
import time

import pymatching
import stim

from soft_info import get_noise_dict_from_backend, get_avgs_from_dict, get_repcode_IQ_map
from soft_info import RepetitionCodeStimCircuit, inv_qubit_mapping, gaussianIQConvertor
from soft_info import get_cols_to_keep, generate_subsets_with_center, get_subsample_layout
from Scratch import load_calibration_memory
from src import cpp_soft_info as csi

file_name  = f'./results/result_day2/infoPerfo{DEVICE}_{state}_{ROUNDS}.json'

HANDLE_OUTLIERS = True

rel_error = 1
_RESETS = False
nb_intervals = -1

# KDE BANDWIDTHS
lin = [0.6, 1.2, 1]
num_points = 20
bandwidths = np.linspace(lin[0], lin[1], lin[2])


for calib_date in jobs_by_calibration_date.keys():
    # get the noise dict of that date
    noise_dict = get_noise_dict_from_backend(provider, DEVICE, date = calib_date)

    # get the KDE of that date

    while True:
        try:
            all_memories, gmm_dict, _ = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                        other_date=calib_date, post_process=True,
                                                                        double_msmt=False)
            break
        except:
            sleep(5)

    kde_dict = csi.get_KDEs(all_memories, bandwidths, relError=rel_error, absError=-1, num_points=num_points)

    while True:
        try:
            all_memories_PS, gmm_dict_PS, msmt_err_dict_PS = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                            other_date=calib_date, post_process=True,
                                                                            double_msmt=True)
            break
        except:
            sleep(5)

    # Get the mean msmt errors
    p_soft_mean = 0
    p_hard_mean = 0
    for key, value in msmt_err_dict_PS.items():
        p_soft_mean += value['p_soft']
        p_hard_mean += value['p_hard']
    p_soft_mean /= len(msmt_err_dict_PS)
    p_hard_mean /= len(msmt_err_dict_PS)
    print(msmt_err_dict_PS, p_soft_mean, p_hard_mean)


    # Retrieve the memories
    memories = []
    for job_id in tqdm(jobs_by_calibration_date[calib_date], desc=f"Retrieving jobs of {calib_date} calibration"):
        d = md[md["job_id"] == job_id]["distance"].values[0] 
        T = md[md["job_id"] == job_id]["rounds"].values[0] # Should be 10

        # Get the job
        job = provider.retrieve_job(job_id)
        memory = job.result().get_memory()
        memories.append(memory)

    # Stack the memories vertically
    big_memory = np.vstack(memories)
    NB_SHOTS = big_memory.shape[0]

    # Get the layout of the last job (same as previous)
    layout_des = job.deserialize_layout(job.initial_layouts()[0]) # only 1 layout
    link_qubits = list(layout_des['link_qubit'].values())
    code_qubits = list(layout_des['code_qubit'].values())

    # Get the pSoft and countMat matrices
    big_layout = link_qubits + code_qubits
    inverted_q_map = inv_qubit_mapping(get_repcode_IQ_map(big_layout, synd_rounds=T))

    pSoft, countMat = csi.iqConvertor(big_memory, inverted_q_map, kde_dict, rel_error, -1,
                                                handleOutliers = HANDLE_OUTLIERS)


    # Subsample decoding
    for D_NEW in tqdm(distances):
        subsets = generate_subsets_with_center(d, D_NEW)

        err_s_K_mean_new = []
        err_s_K_indiv_new = []
        err_s_K_mean_old = []
        err_s_K_indiv_old = []
        for subset in (subsets):             
            cols_to_keep = get_cols_to_keep(subset, T, d)

            # Get the subset of pSoft and countMat
            pSoft_sub = pSoft[:, cols_to_keep]
            countMat_sub = countMat[:, cols_to_keep]

            # Means for hard decoding
            pSoft_mean = np.mean(pSoft_sub)

            # Get the layout
            layout = get_subsample_layout(subset, link_qubits, code_qubits)
            qubit_mapping = get_repcode_IQ_map(layout, synd_rounds=T)

            # Get the noise avgs
            avgs = get_avgs_from_dict(noise_dict, layout)
            noise_list = [avgs["two_gate"], avgs["single_gate"], avgs["t1_err"], avgs["t2_err"]]
            readout = avgs["readout"]
            noise_list += [(p_hard_mean+p_soft_mean), p_hard_mean, p_soft_mean]
            #[twog_err, sglg_err, t1_err, t2_err, readout_err, hard_err, soft_err]

            # Stim models
            subsampling = (D_NEW != d)

            code_indiv_for_soft = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=True, layout=layout,
                                                            msmt_err_dict=msmt_err_dict_PS)
            model_indiv_for_soft = code_indiv_for_soft.circuits[LOGICAL].detector_error_model()

            code_mean_for_soft = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=subsampling, no_fin_soft=True, layout=None,
                                                            msmt_err_dict=None)
            model_mean_for_soft = code_mean_for_soft.circuits[LOGICAL].detector_error_model()            

            # Decoding
            num_err_per_bit_mean_new= []
            num_err_per_bit_mean_old = []
            num_err_per_bit_indiv_new = []
            num_err_per_bit_indiv_old = []
            nb_bits = list(range(1, 20, 1))
            print()
            for nBits in (nb_bits):
                print(f"nBits = {nBits}", end=" ")
                pSoft_trunc_new = csi.quantizeMatrixVectorized(pSoft_sub, nBits)
                pSoft_trunc_old = csi.quantizeMatrixEntrywise(pSoft_sub, nBits)

                res_new_mean = csi.decodeConvertorAll(model_mean_for_soft, countMat_sub, pSoft_trunc_new, T,
                                                      int(LOGICAL), _RESETS, decode_hard=False)
                res_old_mean = csi.decodeConvertorAll(model_mean_for_soft, countMat_sub, pSoft_trunc_old, T,
                                                        int(LOGICAL), _RESETS, decode_hard=False)
                res_new_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMat_sub, pSoft_trunc_new, T,
                                                        int(LOGICAL), _RESETS, decode_hard=False)
                res_old_indiv = csi.decodeConvertorAll(model_indiv_for_soft, countMat_sub, pSoft_trunc_old, T,
                                                        int(LOGICAL), _RESETS, decode_hard=False)
                
                num_err_per_bit_mean_new.append(res_new_mean.num_errors)
                num_err_per_bit_mean_old.append(res_old_mean.num_errors)
                num_err_per_bit_indiv_new.append(res_new_indiv.num_errors)
                num_err_per_bit_indiv_old.append(res_old_indiv.num_errors)
            
            err_s_K_mean_new.append(num_err_per_bit_mean_new)
            err_s_K_indiv_new.append(num_err_per_bit_indiv_new)
            err_s_K_mean_old.append(num_err_per_bit_mean_old)
            err_s_K_indiv_old.append(num_err_per_bit_indiv_old)

            
  
        # Save the results
        old_trunc = {
            "decoding": "trunc_old",
            "d_new": str(D_NEW),
            "nb_bits": nb_bits,
            "error_list_dict": {
                "soft_mean": err_s_K_mean_old,
                "soft_indiv": err_s_K_indiv_old,
            },
            "error_mean_dict": {
                # do the mean over the subsets
                "soft_mean": np.mean(np.array(err_s_K_mean_old), axis=0).tolist(), 
                "soft_indiv": np.mean(np.array(err_s_K_indiv_old), axis=0).tolist(),
            },
            "additional_info": {
                "rel_error": rel_error,
                "pSoft_mean": pSoft_mean,
                "bandwidth_linspace": lin,
                "num_points_bandwidths": num_points,
                "offset": 0.5,
            },
        }

        new_trunc = {
            "decoding": "trunc_new",
            "d_new": str(D_NEW),
            "nb_bits": nb_bits,
            "error_list_dict": {
                "soft_mean": err_s_K_mean_new,
                "soft_indiv": err_s_K_indiv_new,
            },
            "error_mean_dict": {
                # do the mean over the subsets
                "soft_mean": np.mean(np.array(err_s_K_mean_new), axis=0).tolist(),
                "soft_indiv": np.mean(np.array(err_s_K_indiv_new), axis=0).tolist(),
            },
        }


        if not os.path.exists(file_name):
            data = {}
        else:
            with open(file_name, "r") as f:
                data = json.load(f)

        if job_id not in data.keys():
            data[job_id] = [{'totshots': NB_SHOTS}, old_trunc, new_trunc]
        else:
            data[job_id].append(old_trunc)
            data[job_id].append(new_trunc)
        
        with open(file_name, "w") as f:
            json.dump(data, f, indent=4)

17:20:08 Warning: Z0 decoding. Negative T2 error -0.08 % for qubit 38, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
17:20:08 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 39, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
17:20:08 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 111, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 13:11:56.380893+00:00.
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 13:12:25.263713+00:00.
{0: {'p_hard': 0.005111760223520447, 'p_soft': 0.000476250952501905}, 1: {'p_hard': 0.00444500889001778, 'p_soft': 0.0052387

Retrieving jobs of 2024-03-23 13:12:01.375000+00:00 calibration: 100%|██████████| 1/1 [00:02<00:00,  2.54s/it]
  0%|          | 0/8 [00:00<?, ?it/s]

next subset
nBits = 1 nBits = 2 nBits = 3 nBits = 4 nBits = 5 nBits = 6 nBits = 7 nBits = 8 nBits = 9 nBits = 10 nBits = 11 nBits = 12 nBits = 13 nBits = 14 nBits = 15 nBits = 16 nBits = 17 nBits = 18 nBits = 19 next subset
nBits = 1 nBits = 2 nBits = 3 nBits = 4 nBits = 5 nBits = 6 nBits = 7 nBits = 8 nBits = 9 nBits = 10 nBits = 11 nBits = 12 nBits = 13 nBits = 14 nBits = 15 nBits = 16 nBits = 17 nBits = 18 nBits = 19 

 12%|█▎        | 1/8 [01:51<13:02, 111.85s/it]

next subset
nBits = 1 nBits = 2 nBits = 3 nBits = 4 nBits = 5 nBits = 6 nBits = 7 nBits = 8 nBits = 9 nBits = 10 nBits = 11 nBits = 12 nBits = 13 nBits = 14 nBits = 15 nBits = 16 nBits = 17 nBits = 18 nBits = 19 next subset
nBits = 1 nBits = 2 nBits = 3 nBits = 4 nBits = 5 nBits = 6 nBits = 7 nBits = 8 nBits = 9 nBits = 10 nBits = 11 nBits = 12 nBits = 13 nBits = 14 nBits = 15 nBits = 16 nBits = 17 nBits = 18 nBits = 19 

...

 75%|███████▌  | 6/8 [10:07<03:19, 99.57s/it] 

next subset
nBits = 1 nBits = 2 nBits = 3 nBits = 4 nBits = 5 nBits = 6 nBits = 7 nBits = 8 nBits = 9 nBits = 10 nBits = 11 nBits = 12 nBits = 13 nBits = 14 nBits = 15 nBits = 16 nBits = 17 nBits = 18 nBits = 19 next subset
nBits = 1 nBits = 2 nBits = 3 nBits = 4 nBits = 5 nBits = 6 nBits = 7 nBits = 8 nBits = 9 nBits = 10 nBits = 11 nBits = 12 nBits = 13 nBits = 14 nBits = 15 nBits = 16 nBits = 17 nBits = 18 nBits = 19 next subset
nBits = 1 nBits = 2 nBits = 3 nBits = 4 nBits = 5 nBits = 6 nBits = 7 nBits = 8 nBits = 9 nBits = 10 nBits = 11 nBits = 12 nBits = 13 nBits = 14 nBits = 15 nBits = 16 nBits = 17 nBits = 18 nBits = 19 next subset
nBits = 1 nBits = 2 nBits = 3 nBits = 4 nBits = 5 nBits = 6 nBits = 7 nBits = 8 nBits = 9 nBits = 10 nBits = 11 nBits = 12 nBits = 13 nBits = 14 nBits = 15 nBits = 16 nBits = 17 nBits = 18 nBits = 19 next subset
nBits = 1 nBits = 2 nBits = 3 nBits = 4 nBits = 5 nBits = 6 nBits = 7 nBits = 8 nBits = 9 nBits = 10 nBits = 11 nBits = 12 nBits = 13 nBits 

 88%|████████▊ | 7/8 [11:54<01:42, 102.12s/it]

next subset
nBits = 1 nBits = 2 nBits = 3 nBits = 4 nBits = 5 nBits = 6 nBits = 7 nBits = 8 nBits = 9 nBits = 10 nBits = 11 nBits = 12 nBits = 13 nBits = 14 nBits = 15 nBits = 16 nBits = 17 nBits = 18 nBits = 19 next subset
nBits = 1 nBits = 2 nBits = 3 nBits = 4 nBits = 5 nBits = 6 nBits = 7 nBits = 8 nBits = 9 nBits = 10 nBits = 11 nBits = 12 nBits = 13 nBits = 14 nBits = 15 nBits = 16 nBits = 17 nBits = 18 nBits = 19 next subset
nBits = 1 nBits = 2 nBits = 3 nBits = 4 nBits = 5 nBits = 6 nBits = 7 nBits = 8 nBits = 9 nBits = 10 nBits = 11 nBits = 12 nBits = 13 nBits = 14 nBits = 15 nBits = 16 nBits = 17 nBits = 18 nBits = 19 next subset
nBits = 1 nBits = 2 nBits = 3 nBits = 4 nBits = 5 nBits = 6 nBits = 7 nBits = 8 nBits = 9 nBits = 10 nBits = 11 nBits = 12 nBits = 13 nBits = 14 nBits = 15 nBits = 16 nBits = 17 nBits = 18 nBits = 19 next subset
nBits = 1 nBits = 2 nBits = 3 nBits = 4 nBits = 5 nBits = 6 nBits = 7 nBits = 8 nBits = 9 nBits = 10 nBits = 11 nBits = 12 nBits = 13 nBits 

100%|██████████| 8/8 [13:32<00:00, 101.57s/it]
